# Working is still ongoing

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import plotly.graph_objects as go
import plotly.express as ex

In [ ]:
sns.set_theme("paper")
sns.set_style("white")

In [ ]:
pd.set_option( "display.max_rows",120)
pd.set_option( "display.max_columns", 120)

In [ ]:
train_file = "../input/tabular-playground-series-sep-2021/train.csv"
train_master_df = pd.read_csv( train_file ,sep =",")
train_master_df.head()

In [ ]:
print ( " Data frame shape = {}".format(train_master_df.shape ))

# Check data  each column value distribution 

In [ ]:
train_master_df.describe().T

* Have 118 features, and none of feaure are constant,
* Values are linearly increasing

# Check number of claim and non claim count, to see data is imbalanced or not 

In [ ]:
fig = plt.figure( figsize =(8,8), dpi = 90 )
sns.countplot( train_master_df["claim"])
plt.show()

# Check for any missing values in dataframe 

In [ ]:
train_master_df.isnull().sum(axis = 0)

In [ ]:
print( "Max percentage of missing values in dataframe ={}".format( train_master_df.isnull().sum().max()/ train_master_df.shape[0]))

## Above mising percentage might be differenet, taken only max number
## Below Code to check missing values, we can make decission based on final percentage of data 

In [ ]:
print( "by droping all missing values remaining % data ={} %".format(100* train_master_df.dropna(axis =0 ).shape[0]/train_master_df.shape[0] ) )

* ** 37.52% data after removing all missing values, is not a good one we should do simthing better**
* ** Let us do missing values imputation, I would select adding median value of respective column to mising places **
* ** With this we can retain 100% of data **
* ** There may be a question why median, why not mean or min or max, median will take middle value of distribution, were as mean will have high influence by outliers **


In [ ]:
for each_column in train_master_df.columns:
    if each_column not in ["claim","id"] : train_master_df[each_column].fillna( train_master_df[each_column].median(), inplace = True )
train_master_df.isnull().sum()

# With this, now we have 0 missing values, we can use 10% of data

# Visualize claim distribution with respect to other features
# Using pair plot will create 117*117 plots which not suitable for visualization, might very hard to view as well
# Instead of that we can use one feature and see how it is trending with respect to other. toget feel of data distribution of claim and non claim

In [ ]:
'''
# hacountplottal 118 columns, For ploting we can have 10 column and 12 rows
fig = plt.figure( figsize = ( 10, 10 ), dpi = 90  )
counter =1
for each_column in train_master_df.columns:
    if each_column not in ["claim", "id", "f11"]:
        ax = plt.subplot( 12,10, counter )
        sns.scatterplot( x = train_master_df["f11"], y = train_master_df[each_column], ax = ax ,hue = train_master_df["claim"] )
        counter +=1
plt.tight_layout()
plt.show()
'''

* We have 117 features, that is plenty. We can reduce features wich are highly correlating and thos features which doesn't show different grouping
* To remove features we can look at Correlation matrix or use RFE ( Recursive Feature Elimination ) to remove features. This we can set as baseline
* Other way is to do PCA, by doing that we can remove all correlating features & dimensanality reduction as well.

In [ ]:
feature_columns = train_master_df.columns.to_list()
feature_columns.remove("id")
feature_columns.remove("claim")

In [ ]:
corr_matrix = train_master_df[feature_columns].corr()
corr_matrix.round( 2)

In [ ]:
fig = plt.figure( figsize = (25,25), dpi = 90 )
sns.heatmap (corr_matrix.round(2),annot = True,annot_kws={"size":  5 } )
plt.show()

# Good news is, we don't have multi colinearity problem. we can use all features for model 
# Next step is to check is the data points trend, i.e min to max varaition, this give clues for us type of data scalar to use 

In [ ]:
if False :

    fig = plt.figure( figsize = ( 120,120 ), dpi = 100  )
    counter =1
    for each_column in train_master_df.columns:
        if each_column not in ["claim", "id"]:
            plt.subplot( 24,5, counter )
            sns.histplot( x= train_master_df[each_column],hue = train_master_df["claim"] )
            counter +=1
    plt.tight_layout()
    plt.show()

# From plot we can clearly see, have good number of outliers
# We should remove those outliersor clamp those values with certain threshold
# First methos we do is removing outliers, and see how well the model behaves then followed by we can use clamping method

In [ ]:
feature_columns = train_master_df.columns.to_list()
feature_columns.remove("id")
feature_columns.remove("claim")

In [ ]:
def df_plot( df ):
    fig = plt.figure( figsize = ( 120,120 ), dpi = 100  )
    counter =1
    for each_column in train_master_df.columns:
        if each_column not in ["claim", "id"]:
            plt.subplot( 24,5, counter )
            sns.histplot( x= df[each_column],hue = df["claim"] )
            counter +=1
    plt.tight_layout()
    plt.show()

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

In [ ]:
outlier_detector = LocalOutlierFactor( n_neighbors = 1, algorithm = "auto",metric ="l1" ,n_jobs = -1)
data = outlier_detector.fit_predict( train_master_df[ feature_columns[0:-1] ] )

In [ ]:
len( data[data ==-1 ])

In [ ]:
train_master_df_2 = train_master_df[data ==1 ]
train_master_df_2.shape

In [ ]:
fig = plt.figure( figsize = ( 120,120 ), dpi = 100  )
counter =1
for each_column in train_master_df_2.columns:
    if each_column not in ["claim", "id"]:
        plt.subplot( 24,5, counter )
        sns.boxplot( y= train_master_df_2[each_column],x = train_master_df_2["claim"] )
        counter +=1

plt.tight_layout()
plt.show()
#sns.boxplot( y = train_master_df_2[feature_columns[0]], x = train_master_df_2["claim"])
#sns.swarmplot( y = train_master_df_2[feature_columns[0]], x = train_master_df_2["claim"], color = "black")